# DATA GENERATION

IMPORTING NECESSARY DEPENDENCIES

In [5]:
import pandas as pd
import numpy as np
import json
import csv
import os

FUNCTION TO CONVERT JSON FILE INTO CSV FILE:

In [3]:
def json_to_csv(input_file, output_file):
    with open(input_file, 'r') as json_file:
        data = json.load(json_file)

    if not data:
        print("No data found in the JSON file.")
        return

    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        writer.writerow(data[0].keys())

        for row in data:
            writer.writerow(row.values())

The function loads JSON data, checks for emptiness, then writes it into a CSV file, converting dictionaries to rows.

FUNCTION TO READ AND CONVERT THE JSON INTO TABULAR FORMAT:

In [4]:
def read_json_file(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def json_to_tabular(json_data):
    df = pd.json_normalize(json_data)
    return df

FUNCTION TO FORMULATE IMU READINGS:

In [5]:
def ms_imu(file_path):
    json_data=read_json_file(file_path)
    j=json_to_tabular(json_data)
    j[['linear_accel_x', 'linear_accel_y', 'linear_accel_z']] = pd.DataFrame(j['linear_accel'].tolist(), index=j.index)
    j=j.drop(columns=['linear_accel','rotation_rate'])
    j_merged=j.groupby('utime').mean().reset_index()
    return j_merged.iloc[:20]

FUNCTION TO FORMULATE STEERANGLE READINGS:

In [6]:
def steerangle(file_path):
    json_data = read_json_file(file_path)
    
    j=json_to_tabular(json_data)
    j=j.groupby('utime').mean().reset_index()
    return j.iloc[:20]

FUNCTION TO FORMULATE MONITOR READINGS:

In [ ]:
def vehicle_monitor(file_path):
    json_data = read_json_file(file_path)
    j=json_to_tabular(json_data)
    j=j.groupby('utime').mean().reset_index()
    return j.iloc[:20]

FUNCTION TO FORMULATE POSITION READINGS:

In [ ]:
def pose(file_path):
    json_data = read_json_file(file_path)
    j=json_to_tabular(json_data)
    j[['rotation_rate_x', 'rotation_rate_y', 'rotation_rate_z']] = pd.DataFrame(j['rotation_rate'].tolist(), index=j.index)
    j=j.drop(columns=['rotation_rate'])
    j_merged=j.groupby('utime').mean().reset_index()
    return j_merged.iloc[:20]

FUNCTION TO FORMULATE VEHICLE SENSOR READINGS:

In [9]:
def zoe_veh(file_path):
    json_data = read_json_file(file_path)
    j=json_to_tabular(json_data)
    j=j.groupby('utime').mean().reset_index()
    return j.iloc[:20]

FUNCTION TO FORMULATE ZOE SENSOR READINGS:

In [10]:
def zoesensors(file_path):
    json_data = read_json_file(file_path)
    j=json_to_tabular(json_data)
    j=j.groupby('utime').mean().reset_index()
    return j.iloc[:20]

FUNCTION TO SAVE THE FORMULATED FILES IN A DIRECTORY:

In [11]:
def save_directory(df,directory_path,file_name):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    file_path = os.path.join(directory_path, file_name)
    df.to_csv(file_path,index=False)

FUNCTION TO MERGE ALL THE FILES INTO A SINGLE DATAFRAME:

In [48]:
def merged(scene):
    try:
        scene_ms_imu=ms_imu("C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Working datasets/scene-"+str(scene).zfill(4)+"_ms_imu_filtered.json")
        scene_pose=pose("C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Working datasets/scene-"+str(scene).zfill(4)+"_pose_filtered.json")
        scene_sensors=zoesensors("C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Working datasets/scene-"+str(scene).zfill(4)+"_zoesensors_filtered.json")
        scene_zoe_veh=zoe_veh("C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Working datasets/scene-"+str(scene).zfill(4)+"_zoe_veh_info_filtered.json")
        scene_steer=steerangle("C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Working datasets/scene-"+str(scene).zfill(4)+"_steeranglefeedback_filtered.json")
        scene_monitor=vehicle_monitor("C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Working datasets/scene-"+str(scene).zfill(4)+"_vehicle_monitor_filtered.json")
    except FileNotFoundError:
        return
    merged_df=pd.merge(scene_pose,scene_ms_imu, on='utime',how='inner')
    merged_df=pd.merge(merged_df,scene_sensors, on='utime',how='inner')
    merged_df=pd.merge(merged_df,scene_zoe_veh, on='utime',how='inner')
    merged_df=pd.merge(merged_df,scene_steer, on='utime',how='inner')
    merged_df=pd.merge(merged_df,scene_monitor, on='utime',how='inner')
    return merged_df

FUNCTION TO SAVE ALL THE SCENES IN A PARTICULAR DIRECTORY AS A CSV FILE:

In [55]:
def save_scenes(scene):
    try:
        m=merged(scene)
        if m is not None:
            directory_path="C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Scenes compressed"
            file_name='scene'+str(scene).zfill(4)+".csv"
            save_directory(m,directory_path,file_name)
        else:
            print(f"Skipping saving scenes for scene {scene} as merged data is None.")
    except KeyError:
        return

FUNCTION TO FILTER THE NECESSARY ATTRIBUTES FROM THE JSON FILE:

In [31]:
def filter_attributes(file_path,output_directory):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"File '{file_path}' not found. Skipping...")
        return
    necessary_attributes = ["utime", "linear_accel", "rotation_rate",'brake_sensor',
                            'steering_sensor', 'throttle_sensor',"value","brake", 
                            "meanEffTorque", "steering", "steering_speed", "throttle",
                            "vehicle_speed", "FL_wheel_speed", "FR_wheel_speed","RL_wheel_speed","RR_wheel_speed"]

    filtered_data = []
    for entry in data:
        filtered_entry = {}
        for attr in necessary_attributes:
            if attr in entry:
                if attr == 'utime':
                    filtered_entry[attr] = str(entry[attr])[:10]
                else:
                    filtered_entry[attr] = entry[attr]

        filtered_data.append(filtered_entry)
    df = pd.DataFrame(filtered_data)
    filtered_file_path = os.path.join(output_directory, os.path.basename(file_path).replace('.json', '_filtered.json'))
    df.to_json(filtered_file_path, orient='records')

In [23]:
out_dir="C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Working datasets"
file_path="C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0001_ms_imu.json"

These are the input and output directories to create the dataset.

In [32]:
for i in range(1,1111):
    attribute_list=['_ms_imu','_pose','_steeranglefeedback','_vehicle_monitor','_zoe_veh_info','_zoesensors']
    out_dir="C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Working datasets"
    for j in attribute_list:
        file_path="C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-"+str(i).zfill(4)+j+".json"
        filter_attributes(file_path,out_dir)

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0037_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0037_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0037_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0037_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0037_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0037_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0186_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0186_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0186_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0186_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0186_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0186_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0223_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0223_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0223_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0223_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0223_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0223_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0322_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0322_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0322_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0322_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0322_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0322_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0470_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0470_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0470_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0470_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0470_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0470_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

...

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0766_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0766_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0766_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0766_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0766_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0766_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0801_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0801_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0801_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0801_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0801_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0801_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0843_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0843_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0843_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0843_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0843_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0843_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0944_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0944_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0944_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0944_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0944_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0944_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0973_ms_imu.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0973_pose.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0973_steeranglefeedback.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0973_vehicle_monitor.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0973_zoe_veh_info.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/can_bus/can_bus/scene-0973_zoesensors.json' not found. Skipping...
File 'C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artifi

This loop iteratively runs through all the json files available from different datasets and filters out the necessary attributes from the dataset and stores it in a particular directory which was mentioned above.

In [56]:
for i in range(1,1111):
    save_scenes(i)

Skipping saving scenes for scene 37 as merged data is None.
Skipping saving scenes for scene 40 as merged data is None.
Skipping saving scenes for scene 136 as merged data is None.
Skipping saving scenes for scene 137 as merged data is None.
Skipping saving scenes for scene 141 as merged data is None.
Skipping saving scenes for scene 153 as merged data is None.
Skipping saving scenes for scene 156 as merged data is None.
Skipping saving scenes for scene 161 as merged data is None.
Skipping saving scenes for scene 162 as merged data is None.
Skipping saving scenes for scene 163 as merged data is None.
Skipping saving scenes for scene 164 as merged data is None.
Skipping saving scenes for scene 165 as merged data is None.
Skipping saving scenes for scene 166 as merged data is None.
Skipping saving scenes for scene 167 as merged data is None.
Skipping saving scenes for scene 168 as merged data is None.
Skipping saving scenes for scene 169 as merged data is None.
Skipping saving scenes for

The code iteratively runs through all the filtered json files stored in the particular directory and combines the different datasets of a particular scene into single dataframe which is to be stored in a particular directory

In [68]:
directory = "C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Scenes compressed"
for i, filename in enumerate(os.listdir(directory), start=1):
    new_filename = f"scene{i:03d}.csv"
    old_path = os.path.join(directory, filename)
    new_path = os.path.join(directory, new_filename)
    os.rename(old_path, new_path)

There are occurances of missing scenes in the dataframe. Inorder to store the scenes in a proper manner and combine them, all the scenes are renamed according such that there are 975 scenes starting from scene001 to scene975

In [70]:
import pandas as pd
import os

input_directory = "C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Scenes compressed"
output_directory = "C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Dataset"

stacked_dfs = []

for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(input_directory, filename)
        df = pd.read_csv(filepath)
        stacked_dfs.append(df)
        if len(stacked_dfs) == 15:
            stacked_df = pd.concat(stacked_dfs, ignore_index=True)
            output_filename = f"{filename}"
            output_filepath = os.path.join(output_directory, output_filename)
            stacked_df.to_csv(output_filepath, index=False)
            stacked_dfs = []

The input and the output directories are mentioned. The input directory contains 975 dataframes which is to be converted in such a way that a dataframe is created by stacking 15 dataframes together. The loop iteratively runs through all the files and stacks them on upon the other to combine 15 dataframes and store them. Finally, 65 dataframes are obtained in total.

In [71]:
directory = "C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Dataset"
for i, filename in enumerate(os.listdir(directory), start=1):
    new_filename = f"scene{i:03d}.csv"
    old_path = os.path.join(directory, filename)
    new_path = os.path.join(directory, new_filename)
    os.rename(old_path, new_path)

The files which were stacked and stored are renamed for easy access in such a way that are 65 dataframes which are named from scene001 to scene065

An example scene is taken and the attributes are checked 

In [6]:
scene1=pd.read_csv("C:/Users/mbsas/OneDrive/Dokumenty/4th semester/Artificial Intelligence/Lab/Project/Dataset of 65 scenes/scene001.csv")

In [12]:
scene1

,utime,rotation_rate_x,rotation_rate_y,rotation_rate_z,linear_accel_x,linear_accel_y,linear_accel_z,brake_sensor,steering_sensor,throttle_sensor,...,FL_wheel_speed,FR_wheel_speed,RL_wheel_speed,RR_wheel_speed,value,brake,steering,steering_speed,throttle,vehicle_speed
0,1531883530,0.006161,-0.005749,0.319221,-0.041864,1.322542,9.758100,0.170893,0.189161,0.175446,...,122.656380,136.757804,119.927684,134.493115,3.124996,0.0,191.90,65.70,55.0,14.740
1,1531883531,-0.001484,-0.002749,0.374249,0.405604,1.385668,9.756645,0.170959,0.188631,0.213799,...,121.311197,138.072070,117.365282,134.691842,3.589964,0.0,206.35,-0.05,141.5,14.910
2,1531883532,-0.008093,-0.001479,0.352713,0.868282,1.337957,9.786206,0.170997,0.188298,0.245591,...,136.145024,152.175936,132.689821,149.319276,2.969475,0.0,156.20,-121.55,192.0,16.670
3,1531883533,-0.003526,-0.005735,0.092298,0.833338,0.053724,9.852846,0.171000,0.188507,0.243474,...,162.647097,166.928019,161.888574,166.306272,0.647746,0.0,22.60,-83.70,183.5,19.015
4,1531883534,-0.005878,0.008739,0.000413,0.514038,-0.355521,9.723909,0.171009,0.188396,0.208706,...,180.996765,180.812868,180.595194,180.712788,-0.010796,0.0,-0.80,0.00,91.0,20.775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,1531885432,-0.005961,-0.002250,0.002703,-0.457527,0.005283,9.785023,0.170923,0.188289,0.129020,...,165.373776,165.163170,165.281952,165.405367,0.024416,0.0,1.10,-7.00,0.0,18.830
291,1531885433,-0.005522,-0.000338,-0.012516,-0.504520,-0.157800,9.783396,0.172020,0.188407,0.121438,...,157.408564,156.506057,157.208149,156.697111,-0.112979,0.0,-7.90,-3.00,0.0,17.865
292,1531885434,-0.012075,-0.001809,-0.010858,-0.404060,-0.173880,9.761283,0.170834,0.188330,0.150961,...,148.472850,147.640518,148.200132,147.765201,-0.089100,0.0,-3.60,14.90,38.0,16.875
293,1531885435,-0.016050,0.006095,0.006821,-0.086635,-0.291612,9.778028,0.183382,0.188405,0.137603,...,148.630594,148.619642,148.407352,148.565306,0.050421,0.0,3.30,4.95,0.0,17.005


In [8]:
scene1.columns

Index(['utime', 'rotation_rate_x', 'rotation_rate_y', 'rotation_rate_z',
       'linear_accel_x', 'linear_accel_y', 'linear_accel_z', 'brake_sensor',
       'steering_sensor', 'throttle_sensor', 'meanEffTorque', 'FL_wheel_speed',
       'FR_wheel_speed', 'RL_wheel_speed', 'RR_wheel_speed', 'value', 'brake',
       'steering', 'steering_speed', 'throttle', 'vehicle_speed'],
      dtype='object')

A particular scene contains 21 columns which indicates various parameters of a moving vehicle like rotation rate, linear acceleration, steering sensor etc to analyze the occurance of accident to that particular vehicle.

In [9]:
scene1.describe()

,utime,rotation_rate_x,rotation_rate_y,rotation_rate_z,linear_accel_x,linear_accel_y,linear_accel_z,brake_sensor,steering_sensor,throttle_sensor,...,FL_wheel_speed,FR_wheel_speed,RL_wheel_speed,RR_wheel_speed,value,brake,steering,steering_speed,throttle,vehicle_speed
count,2.950000e+02,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000,...,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000,295.000000
mean,1.531974e+09,-0.000278,-0.000637,0.025471,0.041792,-0.134116,9.776653,0.190924,0.190813,0.178318,...,184.743441,185.630505,184.009203,185.279971,0.274028,0.323729,15.515932,-1.144407,75.705085,21.120119
std,3.316489e+05,0.010450,0.007431,0.091192,0.569663,0.431643,0.056797,0.042343,0.008729,0.048850,...,81.432575,80.375096,81.430636,80.477880,0.987502,1.689638,57.217417,33.110531,78.766421,9.193902
min,1.531884e+09,-0.035272,-0.032394,-0.301858,-1.604356,-1.895392,9.556722,0.170686,0.188167,0.121312,...,0.000000,0.000000,0.000000,0.000000,-3.134611,0.000000,-216.400000,-230.100000,0.000000,0.000000
25%,1.531884e+09,-0.005496,-0.004525,-0.016391,-0.401007,-0.395671,9.750750,0.170919,0.188357,0.125199,...,130.813942,134.622182,129.517938,133.666431,-0.085494,0.000000,-4.475000,-5.000000,0.000000,15.187500
50%,1.531884e+09,0.000036,-0.000026,0.000242,0.013582,-0.248491,9.774841,0.171018,0.188410,0.177095,...,188.525700,187.396464,188.580327,187.423569,-0.004545,0.000000,-0.250000,0.000000,61.000000,21.570000
75%,1.531885e+09,0.004842,0.003108,0.027446,0.476714,0.054257,9.799000,0.177235,0.188518,0.213631,...,240.201646,241.810377,239.564476,241.078959,0.201959,0.000000,11.325000,4.900000,126.750000,27.350000
max,1.533201e+09,0.048243,0.023805,0.394851,1.500501,1.852213,10.031864,0.372590,0.224288,0.309770,...,325.467666,324.679953,324.261702,324.415158,5.353225,14.000000,307.450000,130.450000,314.000000,37.015000


In [11]:
scene1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   utime            295 non-null    int64  
 1   rotation_rate_x  295 non-null    float64
 2   rotation_rate_y  295 non-null    float64
 3   rotation_rate_z  295 non-null    float64
 4   linear_accel_x   295 non-null    float64
 5   linear_accel_y   295 non-null    float64
 6   linear_accel_z   295 non-null    float64
 7   brake_sensor     295 non-null    float64
 8   steering_sensor  295 non-null    float64
 9   throttle_sensor  295 non-null    float64
 10  meanEffTorque    295 non-null    float64
 11  FL_wheel_speed   295 non-null    float64
 12  FR_wheel_speed   295 non-null    float64
 13  RL_wheel_speed   295 non-null    float64
 14  RR_wheel_speed   295 non-null    float64
 15  value            295 non-null    float64
 16  brake            295 non-null    float64
 17  steering        

A machine learaning model needs to be trained to check the deviations from the normal and spot the level of intensity of the accidents.
The attributes present in the dataframes can be used to compute the severity of the accident by checking the values of the parameters and indicate the severity